In [2]:
import json
import csv
from urllib.parse import urlparse
from collections import Counter
from scrapper import Scrapper
from IPython.display import clear_output
import pandas as pd
import numpy as np
import sys
import multiprocessing as multi
from multiprocessing import Queue

In [7]:
links = []
with open(r'C:\Users\Anshul\Downloads\newsroom-thin.tar\newsroom-thin\train.jsonl\train-shell.jsonl', 'r') as f:
    for jline, i in zip(f.readlines(), range(100_000)):
        obj = json.loads(jline)
        d = {
            'id': i,
            'link': obj['archive']
        }
        links.append(d)

with open(r'links.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['id','link'])
    writer.writeheader()
    writer.writerows(links)

In [13]:
domains = []
id_domain_list = []
with open(r'links.csv', 'r') as f:
    reader = csv.DictReader(f)
    for each in reader:
        httpsI = each['link'].rfind('https')
        if httpsI == -1 or httpsI == 0:
            httpsI = each['link'].rfind('http')
        link = each['link'][httpsI:]
        domain = urlparse(link).netloc
        id_domain = {
            'id': each['id'],
            'domain': domain
        }
        id_domain_list.append(id_domain)
        domains.append(domain)
domains = Counter(domains)
s = 0
selected_domains = []
for w in sorted(domains, key=domains.get, reverse=True):
    s += domains[w]
    selected_domains.append(w)
    if s >= 60_000:
        break

In [14]:
selected_domains

['www.nytimes.com',
 'www.washingtonpost.com',
 'www.foxnews.com',
 'www.theguardian.com',
 'www.nydailynews.com',
 'www.cnn.com',
 'mashable.com',
 'www.forbes.com',
 'www.wsj.com',
 'time.com',
 'www.cnbc.com',
 'content.usatoday.com',
 'www.usatoday.com',
 'www.9news.com.au',
 'www.tmz.com']

In [28]:
selected_ids = []
with open(r'links.csv', 'r') as f:
    reader = csv.DictReader(f)
    for each in reader:
        httpsI = each['link'].rfind('https')
        if httpsI == -1 or httpsI == 0:
            httpsI = each['link'].rfind('http')
        link = each['link'][httpsI:]
        domain = urlparse(link).netloc
        if domain in selected_domains:
            dict_ = {
                'id': each['id'],
                'link': each['link']
            }
            selected_ids.append(dict_)
df = pd.DataFrame().from_dict(selected_ids)

In [31]:
df.head()

,id,link
0,0,https://web.archive.org/web/2016040819id_/http...
1,1,https://web.archive.org/web/2014072519id_/http...
2,5,https://web.archive.org/web/2007052219id_/http...
3,6,https://web.archive.org/web/2016041519id_/http...
4,8,https://web.archive.org/web/2015090619id_/http...


In [32]:
df.to_csv('selected_links.csv', index=False)

In [7]:
df.head()

,id,link
0,0,https://web.archive.org/web/2016040819id_/http...
1,1,https://web.archive.org/web/2014072519id_/http...
2,5,https://web.archive.org/web/2007052219id_/http...
3,6,https://web.archive.org/web/2016041519id_/http...
4,8,https://web.archive.org/web/2015090619id_/http...


In [31]:
list(range(0,len(df)+1,(len(df))//4))

[0, 15340, 30680, 46020, 61360]

In [ ]:
def chunks(n, page_list):
    """Splits the list into n chunks"""
    return np.array_split(page_list,n)
 

def trgt_fun(df, queue):
    sc = Scrapper()
    x = []
    for index, row in df.iterrows():
        sc.setUrl(row['link'])
        temp = {
            'id': row['id'],
            'text': sc.getArticle()
        }
        x.append(temp)
    res = queue.get()
    res.extend(x)
    queue.put(res)

if __name__ == '__main__':
    df = pd.read_csv('selected_links.csv')
    res = []

    name = 'anshul'
    mem_dict = {
        'anshul':{
            'l': 0,
            'r': 15340
        },
        'timi':{
            'l': 15340,
            'r': 30680
        },
        'piyush':{
            'l': 30680,
            'r': 46020
        },
        'anish':{
            'l': 46020,
            'r': 61361
        },
    }

    df = df.iloc[mem_dict[name]['l']:mem_dict[name]['r']]

    '''
    anshul - 0:15340
    timi - 15340:30680
    piyush - 30680:46020
    anish - 46020:61361
    '''
    cpus = multi.cpu_count()

    workers = []
    page_list = df
    page_bins = chunks(cpus, page_list)
    queue = Queue()
    queue.put(res)
    print('Using {}/{} CPU(s)'.format(cpus,cpus))
    for cpu in range(cpus):
        sys.stdout.write("CPU " + str(cpu) + "\n")
        worker = multi.Process(name=str(cpu), 
                            target=trgt_fun, 
                            args=(page_bins[cpu],queue))
        worker.start()
        workers.append(worker)

    for worker in workers:
        worker.join()

    df1 = pd.DataFrame().from_records(queue.get())
    df1.to_csv('works_{}.csv'.format(name))
